!pip install --quiet easygui
!pip install --quiet dash
!pip install --quiet plotly
!pip install --quiet dash_bootstrap_components
!pip install --quiet jupyter_dash


In [1]:
import plotly.graph_objects as go
import json
import easygui
import dash
from collections import defaultdict
from dash import html, dcc, Input, Output, State, ALL
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from flask import request
#from tkMessageBox import showinfo, showwarning, showerror


In [2]:
null_text = '(not assigned)'

vkey_ids = {
    'Esc':'1',
    '1':'2',
    '2':'3',
    '3':'4',
    '4':'5',
    '5':'6',
    '6':'7',
    '7':'8',
    '8':'9',
    '9':'10',
    '0':'11',
    '-':'12',
    '=':'13',
    '⇐':'14',
    '⇒':'15',
    'Q':'16',
    'W':'17',
    'E':'18',
    'R':'19',
    'T':'20',
    'Y':'21',
    'U':'22',
    'I':'23',
    'O':'24',
    'P':'25',
    '[':'26',
    ']':'27',
    '⤶':'28',
    'LCtl':'29',
    'A':'30',
    'S':'31',
    'D':'32',
    'F':'33',
    'G':'34',
    'H':'35',
    'J':'36',
    'K':'37',
    'L':'38',
    ';':'39',
    '\'':'40',
    '~':'41',
    'L⇑':'42',
    '\\':'43',
    'Z':'44',
    'X':'45',
    'C':'46',
    'V':'47',
    'B':'48',
    'N':'49',
    'M':'50',
    ',':'51',
    '.':'52',
    '/':'53',
    'R⇑':'54',
    'N*':'55',
    'LAlt':'56',
    '⟶⟶':'57',
    'Caps':'58',
    'F1':'59',
    'F2':'60',
    'F3':'61',
    'F4':'62',
    'F5':'63',
    'F6':'64',
    'F7':'65',
    'F8':'66',
    'F9':'67',
    'F10':'68',
    'NP':'69',
    'Scr':'70',
    'N7':'71',
    'N8':'72',
    'N9':'73',
    'N-':'74',
    'N4':'75',
    'N5':'76',
    'N6':'77',
    'N+':'78',
    'N1':'79',
    'N2':'80',
    'N3':'81',
    'N0':'82',
    'N.':'83',
    'F11':'87',
    'F12':'88',
    '⊞':'91',
    'N⤶':'156',
    'RCtl':'157',
    '☰':'161',
    'N/':'181',
    'RAlt':'184',
    'Home':'199',
    '⮝':'200',
    'PgUp':'201',
    '⮜':'203',
    '⮞':'205',
    'End':'207',
    '⮟':'208',
    'PgDn':'209',
    'Ins':'210',
    'Del':'211',
    'Brk':'e11d45e19dc5',
    'Prt':'e02ae037',
    'MouseL': '256',
    'MouseR': '257',
    'MouseC': '258',
    'Mouse3': '259',
    'Mouse4': '260',
    'Mouse5': '261',
    'Mouse6': '262',
    'Mouse7': '263',
    'WheelUp': '264',
    'WheelDn': '265',
}
vkey_names = {vkey_ids[id]:id for id in vkey_ids.keys()}

In [3]:
filename = 'VKeyData.json'

def read_data():
    try:
        with open(filename, 'r') as openfile:
            # Reading from json file
            json_object = json.load(openfile)
        print(json_object)
        print(type(json_object))
    except:
        print(f'{filename} doesn\'t exist.')
        json_object = {}
        json_object[vkey_ids['Esc']] = 'Open Menu'
        json_object[vkey_ids['Ins']] = '+Shift   Display Tweaks Menu\n+Control Upscaler Menu\n+Alt     Upscaler Toggle'
    return json_object

json_object = read_data()
#print(json_object)
#print(type(json_object))

def write_data(json_object):
    json_object = {k:t for k,t in json_object.items() if not is_null_str(t)}
    
    try:
        with open(filename, 'w') as openfile:
            json.dump(json_object, openfile, indent=4)
    except Exception as ex:
        easygui.exceptionbox(ex)

def is_null_str(s):
    return s == None or s.isspace() or len(s) == 0

{'1': 'Menu', '210': 'Select\nDisplay Tweaks: (Shift) Menu\nSexlab/Zaz: Select actor\nAroused: Status\nAroused: (Hold) masturbate', '78': 'SMP Me\n3BA: Toggle SMP (player)\n\nOStim: speed up', '74': 'SMPNPC\n3BA: Toggle SMP (NPCs)\n\nOStim: speed down', '199': 'Chest\nPlace NFF chest', '39': 'Body\nAutobody: Select preset for target.', '70': 'Wings\nHold: Unfurl wings', '49': 'Status\nMilk Mod Economy: Status\nMilk Mod Economy: (Hold) Self-milking', '34': 'Yamete Kudasai: Assault', '33': 'POV', '36': 'Eat\nAnimated Eating: Eat\nAnimated Eating: (CTRL) Drink', '37': 'TFCk\nAnimated Eatings: TFCk', '260': 'EWM\nEasyWheel Menu: Display', '264': 'Next\nEasyWheel Menu: Next\n', '265': 'Prev\nEasyWheel Menu: Previous', '88': 'ENB\nENB: (Shift) Toggle ENB', '261': 'Kill\nValhalla: Execution', '47': 'Target\nTrue Directional Movement: Target Lock', '82': 'SLS\nSexlab Survival: All in One Menu', '27': 'Helmet\nRead the Room: Toggle Helmet', '79': '\nOStim: pullout', '157': 'Cum\nOCum: Show Cum 

In [4]:
#json_object.items()

In [5]:
app = JupyterDash(__name__)

Utility functions for creating the html buttons of the table.

In [6]:
def produce_button(key):
    '''
    Produce a button for the specified key. If it corresponds to a set key in json_object
    then the button will be bold+italics, and include the first six letters of the first line
    of text.
    '''
    global json_object
    kid = vkey_ids[key]

    if kid in json_object.keys(): 
        desc = json_object[kid][0:6].split('\n')[0]
        text = html.Em(html.B(html.Pre(f"{key}\n{desc}")))
        text = html.P(text)

    else:
        desc = '      '
        text = html.Pre(f"{key}\n{desc}")
        text = html.P(text)

    button = html.Button(
        children=text, 
        id={'type':'vkey', 'index':kid}, 
        style={
            'class':'vkey',
            'width':'100%',
        })

    return button

In [7]:
def produce_cell(key, colSpan=1, rowSpan=1):
    '''
    Produce a cell for the keyboard. Each appended → or ↓ represents an increase
    of colspan/rowspace of 1.
    
    The code || represents an empty cell.
    '''
    global buttons
    
    if key == '||': return html.Td('   ', style={'white-space': 'pre'})
    
    elif len(key)>1 and key[-1] == '→':
        key = key[:len(key)-1]
        return produce_cell(key, colSpan+1, rowSpan)
    
    elif len(key)>1 and key[-1] == '↓':
        key = key[:len(key)-1]
        return produce_cell(key, colSpan, rowSpan+1)
    
    elif key not in vkey_ids.keys():
        raise Exception(f'INVALID KEY: {key}')
        
    else: 
        button = produce_button(key)
        buttons[vkey_ids[key]] = button
        return html.Td(button, id=f'cell_{key}', colSpan=colSpan, rowSpan=rowSpan)


In [8]:
def produce_cells(keys_code):
    '''
    Produce a row of cells by splitting up a key string.
    '''
    keys = keys_code.split(' ')
    return [produce_cell(key) for key in keys]

Produce the UI.

In [9]:
buttons = {}
modified_status = html.P('', id='modified_status')
displaying_id = 0
id_assigned = False
run = 0

In [10]:
app.layout = html.Div(id='toplevel', children=[
    html.Table([
        html.Tr([
            html.Th('Main Row', colSpan=16),
            html.Th('Navigation', colSpan=5),
            html.Th('Num Pad', colSpan=4),
            html.Th('Mouse', colSpan=6),
        ]),
        
        html.Tr(produce_cells('Esc || F1 F2 F3 F4 || F5 F6 F7 F8 || F9 F10 F11 F12 || Prt Scr Brk')),
        html.Tr(style={'height':'40px'}),
        html.Tr(produce_cells('~ || 1 2 3 4 5 6 7 8 9 0 - = ⇐→ || Ins Home PgUp || NP N/ N* N- || || || MouseL↓ || MouseR↓')),
        html.Tr(produce_cells('⇒→ Q W E R T Y U I O P [ ] \ || || Del End PgDn || N7 N8 N9 N+↓ || || Mouse5 WheelUp')),
        html.Tr(produce_cells('Caps→ A S D F G H J K L ; \' ⤶→ || || || || || || N4 N5 N6 || || Mouse4 || MouseC')),
        html.Tr(produce_cells('L⇑→ Z X C V B N M , . / R⇑→ || || || || ⮝ || || N1 N2 N3 N⤶↓ || || || || WheelDn')),
        html.Tr(produce_cells('LCtl ⊞ LAlt ⟶⟶→→→→→→→ RAlt ☰ RCtl || || || ⮜ ⮟ ⮞ || N0→ N.')),
    ]),
    html.Div(id='outbox', style={}, children=[
        html.Div(id='data_heading'),
        dcc.Textarea(id='data_display', value='PLACEHOLDER', style={'width':'50%', 'height':'150px'}),
    ]),
    html.Div(modified_status),
])



In [11]:
@app.callback(
    Output('data_heading', 'children'), 
    Output('data_display', 'value'),
    Input({'type': 'vkey', 'index': ALL}, 'n_clicks'), 
)
def update_output(vkey_n_clicks):
    try:
        global id_assigned
        global displaying_id
        global json_object
        
        if dash.ctx.triggered_id != None:
            kid = dash.ctx.triggered_id['index']
            
            if kid in json_object.keys(): text = json_object[kid]
            else: text = null_text
            
            if kid in vkey_names.keys(): name = vkey_names[kid]
            else: name = 'No key selected'

            #print(f'update_output --- {kid}:({type(kid)}) "{name}" = {text}')
            id_assigned = kid in json_object.keys()
            displaying_id = kid
            modified_status.children = ''
            return [html.H3(name), text]
        else:
            name = 'errant call 1'
            text = 'don\'t worry about it'
            return [html.H3(name), text]
        
    except Exception as ex:
        #print('===EXCEPTION===')
        #print(ex)
        #modified_status.children = 'ERROR'
        name = 'errant call 2'
        text = 'worry about it'
        return [html.H3(name), text]


In [12]:
@app.callback(
    Output('modified_status', 'children'), 
    Input('data_display', 'value'))
def text_monitor(text):
    try:
        global id_assigned
        global displaying_id
        global vkey_names
        global buttons
        global json_object
        
        if displaying_id not in vkey_names.keys(): return 'No key selected'
        name = vkey_names[displaying_id]
        
        if displaying_id in json_object.keys(): stored = json_object[displaying_id]
        else: stored = null_text

        #print(f'text_monitor --- {id_assigned} {displaying_id} "{stored}" "{text}"')

        if is_null_str(text): 
            del json_object[displaying_id]
            id_assigned = False
            write_data(json_object)
            buttons[displaying_id].children = []
            buttons[displaying_id].children = html.P(name)
            return '✓'
        else: 
            json_object[displaying_id] = text
            id_assigned = True
            write_data(json_object)
            buttons[displaying_id].children = []
            buttons[displaying_id].children = html.P(html.B(name))
            return '✓'
       
    except Exception as ex:
        #print('===EXCEPTION===')
        #print(ex)
        return '✗'


In [13]:
if __name__=='__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/


<a href="http://localhost:8050/">http://localhost:8050/</a>

In [14]:
import webbrowser

# generate an URL
url = 'http://localhost:8050/'
webbrowser.open(url)

True